# One hot save and clean

In [1]:
import pandas as pd
import csv
import os
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from start import data_path

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
doi_df = pd.read_csv(os.path.join(data_path, 'doi_dates_and_exemptions.csv'))
doi_df.head()

,Unnamed: 0,title,link,p_innovation,possible_laws,text,Unnamed: 0.1,term_phrase,term_year,finalize_phrase,finalize_year,finalize_month,doi_year
0,0,Abbott ISD,https://www.abbottisd.org/ourpages/auto/2018/3...,0.999949,"[25.036, 21.458, 21.451, 21.458, 25.036, 25.11...",1 Abbott Independent School District District ...,0,for the next five years (March 2018-February 2...,2018.0,Abbott Independent School District District of...,2018.0,March,2018.0
1,1,Abernathy ISD,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,0.998340,"[28.216, 25.113, 25.113, 37.0012, 25.092, 28.2...",Abernathy ISD District of Innovation Plan – Dr...,1,of Innovation Plan â Draft Introduction Hous...,2017.0,"the Texas Education Code. On Thursday, January...",2017.0,January,2017.0
2,2,Abilene ISD,https://www.abileneisd.org/wp-content/uploads/...,0.999955,"[21.003, 25.0811, 25.0811, 21.053, 25.0811, 21...",0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,2,ABILENE INDEPENDENT SCHOOL DISTRICT Local Inno...,2018.0,"the AISD Board of Trustees April 9, 2018 1 Boa...",2018.0,April,2018.0
3,3,Academy ISD,https://4.files.edl.io/1a8f/06/29/18/204245-44...,0.999836,"[37.0012, 21.003, 21.053, 25.0811, 21.057, 21....",District of Innovation Plan Developed in coope...,3,2018-2019 school year and concluding at the en...,2018.0,"regularly scheduled board meeting on June 28, ...",2018.0,June,2018.0
4,4,Adrian ISD,http://www.adrianisd.net/UserFiles/Servers/Ser...,0.999945,"[21.003, 25.0812, 21.401, 21.003, 25.081, 21.1...",Adrian ISD District of Innovation Plan House B...,4,term of up to five years beginning June 2018 a...,2018.0,NaN,NaN,NaN,2018.0


## Save create one-hot variables for laws

In [3]:
def make_list(string):
    return [i.replace('[','').replace(']','') for i in string.split(', ')]

doi_df['possible_laws'] = doi_df['possible_laws'].apply(make_list)

In [4]:
mlb = MultiLabelBinarizer()
plans_onehot = doi_df.join(pd.DataFrame(mlb.fit_transform(doi_df.pop('possible_laws')),
                          columns=mlb.classes_,
                          index=doi_df.index))
plans_onehot.head(10)

,Unnamed: 0,title,link,p_innovation,text,Unnamed: 0.1,term_phrase,term_year,finalize_phrase,finalize_year,...,545.207,55.0811,551.001,552.023,552.024,7.056,75.600,921.406,97.1001,97.1055
0,0,Abbott ISD,https://www.abbottisd.org/ourpages/auto/2018/3...,0.999949,1 Abbott Independent School District District ...,0,for the next five years (March 2018-February 2...,2018.0,Abbott Independent School District District of...,2018.0,...,0,0,0,0,0,0,0,0,0,0
1,1,Abernathy ISD,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,0.998340,Abernathy ISD District of Innovation Plan – Dr...,1,of Innovation Plan â Draft Introduction Hous...,2017.0,"the Texas Education Code. On Thursday, January...",2017.0,...,0,0,0,0,0,0,0,0,0,0
2,2,Abilene ISD,https://www.abileneisd.org/wp-content/uploads/...,0.999955,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,2,ABILENE INDEPENDENT SCHOOL DISTRICT Local Inno...,2018.0,"the AISD Board of Trustees April 9, 2018 1 Boa...",2018.0,...,0,0,0,0,0,0,0,0,0,0
3,3,Academy ISD,https://4.files.edl.io/1a8f/06/29/18/204245-44...,0.999836,District of Innovation Plan Developed in coope...,3,2018-2019 school year and concluding at the en...,2018.0,"regularly scheduled board meeting on June 28, ...",2018.0,...,0,0,0,0,0,0,0,0,0,0
4,4,Adrian ISD,http://www.adrianisd.net/UserFiles/Servers/Ser...,0.999945,Adrian ISD District of Innovation Plan House B...,4,term of up to five years beginning June 2018 a...,2018.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,5,Agua Dulce ISD,https://tx02206063.schoolwires.net/cms/lib/TX0...,0.999955,Microsoft Word - DOI 2017-2022.docx Agua Dulce...,5,. Agua Dulce ISD District of Innovation Plan 2...,2017.0,"of the proposed plan. June 12, 2017 â The Ag...",2017.0,...,0,0,0,0,0,0,0,0,0,0
6,6,Alamo Heights ISD,http://www.ahisd.net/UserFiles/Servers/Server_...,0.997152,1 | P a g e Mission Statement The Alamo Height...,6,"a new teacher appraisal system in 2016-2017, c...",2016.0,"artifacts, and other data (see â2016 Engaged...",2016.0,...,0,0,0,0,0,0,0,0,0,0
7,7,Albany ISD,http://www.albanyisd.net/uploads/4/4/4/1/44419...,0.999931,Microsoft Word - innovation_plan.docx Albany I...,7,The Local Innovation Plan Committee met Februa...,2017.0,"Local Innovation Plan at the March 20, 2017 re...",2017.0,...,0,0,0,0,0,0,0,0,0,0
8,8,Aldine ISD,http://insidealdine.com/wp-content/uploads/201...,0.999702,Aldine District of Innovation Plan Approved by...,8,of teacher practice. During the 2016-2017 scho...,2017.0,"November 15, 2016 Appoint Committee October 11...",2017.0,...,0,0,0,0,0,0,0,0,0,0
9,9,Aledo ISD,https://docs.google.com/viewerng/viewer?url=ht...,0.143208,ALEDO\nIndependent School District\nA Past to ...,9,NaN,NaN,NaN,2019.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
drop_cols = [c for c in plans_onehot.columns if c.lower()[:7] == 'unnamed']
plans_onehot = plans_onehot.drop(drop_cols, axis=1)
plans_onehot.head()

,title,link,p_innovation,text,term_phrase,term_year,finalize_phrase,finalize_year,finalize_month,doi_year,...,545.207,55.0811,551.001,552.023,552.024,7.056,75.600,921.406,97.1001,97.1055
0,Abbott ISD,https://www.abbottisd.org/ourpages/auto/2018/3...,0.999949,1 Abbott Independent School District District ...,for the next five years (March 2018-February 2...,2018.0,Abbott Independent School District District of...,2018.0,March,2018.0,...,0,0,0,0,0,0,0,0,0,0
1,Abernathy ISD,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,0.998340,Abernathy ISD District of Innovation Plan – Dr...,of Innovation Plan â Draft Introduction Hous...,2017.0,"the Texas Education Code. On Thursday, January...",2017.0,January,2017.0,...,0,0,0,0,0,0,0,0,0,0
2,Abilene ISD,https://www.abileneisd.org/wp-content/uploads/...,0.999955,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,ABILENE INDEPENDENT SCHOOL DISTRICT Local Inno...,2018.0,"the AISD Board of Trustees April 9, 2018 1 Boa...",2018.0,April,2018.0,...,0,0,0,0,0,0,0,0,0,0
3,Academy ISD,https://4.files.edl.io/1a8f/06/29/18/204245-44...,0.999836,District of Innovation Plan Developed in coope...,2018-2019 school year and concluding at the en...,2018.0,"regularly scheduled board meeting on June 28, ...",2018.0,June,2018.0,...,0,0,0,0,0,0,0,0,0,0
4,Adrian ISD,http://www.adrianisd.net/UserFiles/Servers/Ser...,0.999945,Adrian ISD District of Innovation Plan House B...,term of up to five years beginning June 2018 a...,2018.0,NaN,NaN,NaN,2018.0,...,0,0,0,0,0,0,0,0,0,0


### Rename exemptions so they start with reg

In [6]:
plans_onehot.columns = ['reg' + str(i).replace('.', '_') for i in plans_onehot.columns]
plans_onehot = plans_onehot.drop(['reg'], axis=1)
plans_onehot = plans_onehot.rename(index=str, columns= {'regtitle':'district', 
                                                        'reglink': 'link',
                                                        'regtext': 'text',
                                                        'regp_innovation':'p_doi',                                                        'regterm_year': 'term_year',
                                                        'regterm_phrase': 'term_phrase',
                                                        'regterm_year': 'term_year',
                                                        'regfinalize_phrase': 'finalize_phrase',
                                                        'regfinalize_year': 'finalize_year',
                                                        'regfinalize_month': 'finalize_month',
                                                        'regdoi_year': 'doi_year'})
plans_onehot.head(10)

,district,link,p_doi,text,term_phrase,term_year,finalize_phrase,finalize_year,finalize_month,doi_year,...,reg545_207,reg55_0811,reg551_001,reg552_023,reg552_024,reg7_056,reg75_600,reg921_406,reg97_1001,reg97_1055
0,Abbott ISD,https://www.abbottisd.org/ourpages/auto/2018/3...,0.999949,1 Abbott Independent School District District ...,for the next five years (March 2018-February 2...,2018.0,Abbott Independent School District District of...,2018.0,March,2018.0,...,0,0,0,0,0,0,0,0,0,0
1,Abernathy ISD,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,0.998340,Abernathy ISD District of Innovation Plan – Dr...,of Innovation Plan â Draft Introduction Hous...,2017.0,"the Texas Education Code. On Thursday, January...",2017.0,January,2017.0,...,0,0,0,0,0,0,0,0,0,0
2,Abilene ISD,https://www.abileneisd.org/wp-content/uploads/...,0.999955,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,ABILENE INDEPENDENT SCHOOL DISTRICT Local Inno...,2018.0,"the AISD Board of Trustees April 9, 2018 1 Boa...",2018.0,April,2018.0,...,0,0,0,0,0,0,0,0,0,0
3,Academy ISD,https://4.files.edl.io/1a8f/06/29/18/204245-44...,0.999836,District of Innovation Plan Developed in coope...,2018-2019 school year and concluding at the en...,2018.0,"regularly scheduled board meeting on June 28, ...",2018.0,June,2018.0,...,0,0,0,0,0,0,0,0,0,0
4,Adrian ISD,http://www.adrianisd.net/UserFiles/Servers/Ser...,0.999945,Adrian ISD District of Innovation Plan House B...,term of up to five years beginning June 2018 a...,2018.0,NaN,NaN,NaN,2018.0,...,0,0,0,0,0,0,0,0,0,0
5,Agua Dulce ISD,https://tx02206063.schoolwires.net/cms/lib/TX0...,0.999955,Microsoft Word - DOI 2017-2022.docx Agua Dulce...,. Agua Dulce ISD District of Innovation Plan 2...,2017.0,"of the proposed plan. June 12, 2017 â The Ag...",2017.0,June,2017.0,...,0,0,0,0,0,0,0,0,0,0
6,Alamo Heights ISD,http://www.ahisd.net/UserFiles/Servers/Server_...,0.997152,1 | P a g e Mission Statement The Alamo Height...,"a new teacher appraisal system in 2016-2017, c...",2016.0,"artifacts, and other data (see â2016 Engaged...",2016.0,December,2016.0,...,0,0,0,0,0,0,0,0,0,0
7,Albany ISD,http://www.albanyisd.net/uploads/4/4/4/1/44419...,0.999931,Microsoft Word - innovation_plan.docx Albany I...,The Local Innovation Plan Committee met Februa...,2017.0,"Local Innovation Plan at the March 20, 2017 re...",2017.0,March,2017.0,...,0,0,0,0,0,0,0,0,0,0
8,Aldine ISD,http://insidealdine.com/wp-content/uploads/201...,0.999702,Aldine District of Innovation Plan Approved by...,of teacher practice. During the 2016-2017 scho...,2017.0,"November 15, 2016 Appoint Committee October 11...",2017.0,April,2017.0,...,0,0,0,0,0,0,0,0,0,0
9,Aledo ISD,https://docs.google.com/viewerng/viewer?url=ht...,0.143208,ALEDO\nIndependent School District\nA Past to ...,NaN,NaN,NaN,2019.0,January,2019.0,...,0,0,0,0,0,0,0,0,0,0


# Save

In [7]:
plans_onehot.to_csv((os.path.join(data_path, 'doi_final_wtext.csv')))
plans_onehot = plans_onehot.drop(['text'], axis = 1)
plans_onehot.to_csv((os.path.join(data_path, 'doi_final.csv')))
plans_onehot.to_csv((os.path.join('/Users/kylieleblancKylie/domino/dofis/data/clean', 'doi_final.csv')))

In [4]:
df = pd.read_csv((os.path.join(data_path, 'doi_final.csv')))

In [7]:
df[['district', 'link', 'reg25_0811']]

,district,link,reg25_0811
0,Abbott ISD,https://www.abbottisd.org/ourpages/auto/2018/3...,0
1,Abernathy ISD,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,1
2,Abilene ISD,https://www.abileneisd.org/wp-content/uploads/...,1
3,Academy ISD,https://4.files.edl.io/1a8f/06/29/18/204245-44...,1
4,Adrian ISD,http://www.adrianisd.net/UserFiles/Servers/Ser...,1
5,Agua Dulce ISD,https://tx02206063.schoolwires.net/cms/lib/TX0...,1
6,Alamo Heights ISD,http://www.ahisd.net/UserFiles/Servers/Server_...,1
7,Albany ISD,http://www.albanyisd.net/uploads/4/4/4/1/44419...,1
8,Aldine ISD,http://insidealdine.com/wp-content/uploads/201...,1
9,Aledo ISD,https://docs.google.com/viewerng/viewer?url=ht...,1


In [9]:
df.reg25_0811.mean()

0.9223300970873787

In [19]:
df.iloc[0, 2]
#df[df.district == 'Abbott ISD'].link

'https://www.abbottisd.org/ourpages/auto/2018/3/19/46929646/Abbott%20ISD%20DOI%20Plan%20Final.pdf'

In [15]:
pd.display.max_colwidth = 100

AttributeError: module 'pandas' has no attribute 'display'